# Get a list of all my Spotify playlists

Spotipy documentation: https://spotipy.readthedocs.io/en/latest/

Spotify API documentation: https://developer.spotify.com/web-api/

In [1]:
import spotipy, spotipy.util as util, pandas as pd, math, time
from keys import spotify_username, spotify_client_id, spotify_client_secret, spotify_redirect_uri

In [2]:
limit = 50 #max playlists you can retrieve in a single request
pause = 0.5 #how long to pause between requests

## Authorize spotipy and connect

In [3]:
# define scope to work with playlists then get an auth token
scope = 'playlist-read-private playlist-modify-private playlist-modify-public'
token = util.prompt_for_user_token(spotify_username, scope, spotify_client_id, spotify_client_secret, spotify_redirect_uri)

In [4]:
# open a connection to spotify
sp = spotipy.Spotify(auth=token)

In [5]:
# get the user id associated with this username
user = sp.user(spotify_username)
user_id = user['id']

## Get my own playlists

In [6]:
def get_playlists(user_id):
    
    response = sp.user_playlists(user=user_id, limit=limit, offset=0)
    playlists = response['items']
    playlists_count = response['total']
    additional_call_count = math.ceil((playlists_count - limit) / limit)
    
    for n in range(additional_call_count):
        time.sleep(pause)
        offset = (n + 1) * limit
        playlists_batch = sp.user_playlists(user=user_id, limit=limit, offset=offset)
        playlists.extend(playlists_batch['items'])

    print('Retrieved {} playlists via {} API calls'.format(len(playlists), additional_call_count + 1))
    return playlists

In [7]:
def make_df(playlists):
    df = pd.DataFrame(playlists)
    drop_cols = ['external_urls', 'href', 'images', 'snapshot_id', 'uri']
    df = df[[col for col in df.columns if col not in drop_cols]]
    df['owner'] = df['owner'].map(lambda x: x['id'])
    df['tracks'] = df['tracks'].map(lambda x: x['total'])
    return df

In [8]:
playlists = get_playlists(user_id)

Retrieved 142 playlists via 3 API calls


In [9]:
df = make_df(playlists)
df.head()

,collaborative,id,name,owner,public,tracks,type
0,False,1MQJHcaXE6KiefNSZksfzI,Discover Weekly,spotifydiscover,False,30,playlist
1,False,6ptGZf55DyKdPlVWS3ynVt,Starred,geoff.boeing,True,264,playlist
2,False,2SdIJ48fQqXXDcjgx4aAY8,Wedding,geoff.boeing,False,68,playlist
3,False,0HcttjMuy60QObMqg4Gt5X,African,geoff.boeing,False,196,playlist
4,False,0ec3auWRtoz6gJ7gGIrvk0,Japanese,geoff.boeing,True,309,playlist
